# Post-wildfire vegetation regrowth analysis
Author: Stepan Bryleev

### Moisture Stress Index (MSI) 

In this Jupyter Notebook there is a JavaScript code for calculating and displaying Moisture Stress Index (MSI) values for the study area in GEE using NEON reflectance data.

* Moisture Stress Index is used for canopy stress analysis and productivity prediction.
* $MSI=\frac{MidIR}{NIR}$
* NEON Spectrometer bands B244 (1600 nm) and B088 (819 nm) are used for MidIR and NIR respectively.

```js
// ======================================================= //
// This part of the project is for creating GRSM MSI maps //
// ======================================================= //


// Specify center location and for GRSM
var SiteCenterPoint = ee.Geometry.Point([-83.5, 35.7]);

// Center the map on our area of interest and set zoom level
Map.setCenter(-83.5, 35.7, 12);

var study_polygon = /* color: #98ff00 */ee.Geometry.Polygon(
        [[[-83.5425714556907, 35.701602862979236],
          [-83.5422281329368, 35.690310616971246],
          [-83.53879490539774, 35.63828989061714],
          [-83.5037759844993, 35.615406781430224],
          [-83.48592430887022, 35.61938453875363],
          [-83.47459355041727, 35.627546807436595],
          [-83.47922808124717, 35.65117980654369],
          [-83.47738302453864, 35.682051678012314],
          [-83.4774257678571, 35.69326203209512],
          [-83.47751235961914, 35.71456632200802],
          [-83.47837061486136, 35.7182597023649],
          [-83.48043060302734, 35.72055934657099],
          [-83.53656387329102, 35.70494866628485]]]);

// Define the GRSM perim variable
var grsm_boundary = ee.FeatureCollection('projects/ee-stbr4432/assets/grsm_polygon');

// Define the fire perimeter variable
var fire_perimeter = ee.FeatureCollection('projects/ee-stbr4432/assets/chimney_tops_perim');
var fireBoundGeom = fire_perimeter.geometry();

// Apply the intersection method to the Polygon object.
var polygonIntersection = fireBoundGeom.intersection(grsm_boundary);
print('polygon intersection',polygonIntersection);

// Create study area.
var studyArea = study_polygon.intersection(polygonIntersection);

// Print polygon area in square kilometers.
print('Study area square: ', studyArea.area().divide(1000 * 1000));

// Print polygon perimeter length in kilometers.
print('Study area perimeter: ', studyArea.perimeter().divide(1000));


// ===== Create a cloud masking function ===== //

// Define a function
function neon_mask(image) {
  // Create a single band Weather Quality QA layer & Mask out cloudy pixels from the image
  var weather_qa = image.select(['Weather_Quality_Indicator']);
  var clear_qa = weather_qa.gt(0);
  var nis_image_clear = image.updateMask(clear_qa);
  return nis_image_clear;
}


// === Create a function to calculate and display MSI image with a date === //

// MSI = MidIR / NIR = p1599/p819

// MidIR = NIS Band B244 (1600 nm)
// NIR = NIS Band B088 (819 nm)

// Define a function and bands of interest
function calc_msi(image, date) {
  // NIR = NIS Band B088, MidIR = NIS Band B244
  var nis_image_nir = image.select(["B088"]);
  var nis_image_midir = image.select(["B244"]);
  // Calculate MSI
  var nis_msi =  nis_image_midir.divide(nis_image_nir);
  
  // Display the result (optional)
  // var viz_params = {min:0, max:4, palette:['darkgreen', 'green', 'white', 'pink']};
  // Map.addLayer(nis_msi, viz_params, "GRSM MSI " + date, 0);
  
  return nis_msi;
}


// === Create a function to plot histogram charts === //

// Define a function
function plot_hist(msi_image, studyArea, scale, title) {
  var msi_hist = ui.Chart.image.histogram({image: msi_image, region: studyArea, scale: scale})
      .setOptions({title: title,
            hAxis: {title: 'MSI Value',titleTextStyle: {italic: false, bold: true},},
            vAxis: {title: 'Count', titleTextStyle: {italic: false, bold: true}},});
            
  print(msi_hist);
}

// DEFINE DATES OF INTEREST:

var date_2016 = '2016-06-08';
var date_2017 = '2017-10-05'; 
var date_2021 = '2021-06-18';


//  DATA COLLECTION: 'projects/neon-prod-earthengine/assets/DP3-30006-001'

// ======= Read 2016 data ======= //

// Read in the 2016 SDR Image for GRSM
var NISimage_2016 = ee.Image('projects/neon-prod-earthengine/assets/DP3-30006-001/2016_GRSM_2_SDR')
                      .clip(studyArea);
// Apply masking function to 2016 image
var nis_image_2016_masked = neon_mask(NISimage_2016);


// ======= Read 2017 data ======= //

// Read in the 2017 SDR Image for GRSM
var NISimage_2017 = ee.Image('projects/neon-prod-earthengine/assets/DP3-30006-001/2017_GRSM_3_SDR')
                      .clip(studyArea);
// Apply masking function to 2017 image
var nis_image_2017_masked = neon_mask(NISimage_2017);


// ======= Read 2021 data ======= //

// Read in the 2021 SDR Image for GRSM
var NISimage_2021 = ee.Image('projects/neon-prod-earthengine/assets/DP3-30006-001/2021_GRSM_5_SDR')
                      .clip(studyArea);
// Apply masking function to 2021 image
var nis_image_2021_masked = neon_mask(NISimage_2021);


// ======= Calculate and plot MSI for each date of interest ======= //

var pre_fire_msi_2016 = calc_msi(nis_image_2016_masked, date_2016);
var post_fire_msi_2017 = calc_msi(nis_image_2017_masked, date_2017);
var post_fire_msi_2021 = calc_msi(nis_image_2021_masked, date_2021);


// ======= Create MSI Classification and add MSI images to the map ======= //

// SLD = Styled Layer Descriptor

// Define an SLD style of discrete intervals to apply to the image.
var sld_intervals =
  '<RasterSymbolizer>' +
    '<ColorMap type="intervals" extended="false" >' + 
      '<ColorMapEntry color="#006400" quantity="0.2" label="0.2" />' + // DarkGreen
      '<ColorMapEntry color="#228B22" quantity="0.4" label="0.4" />' + // ForestGreen
      '<ColorMapEntry color="#32CD32" quantity="0.6" label="0.6" />' + // LimetGreen
      '<ColorMapEntry color="#7FFFD4" quantity="0.8" label="0.8" />' + // Aquamarine
      '<ColorMapEntry color="#ADFF2F" quantity="1" label="1" />' + // GreenYellow
      '<ColorMapEntry color="#FFFFE0" quantity="1.2" label="1.2" />' + // LightYellow
      '<ColorMapEntry color="#FFD700" quantity="1.4" label="1.4" />' + // Gold
      '<ColorMapEntry color="#FFC0CB" quantity="1.6" label="1.6" />' + // Pink
      '<ColorMapEntry color="#FF69B4" quantity="1.8" label="1.8" />' + // HotPink
      '<ColorMapEntry color="#FF1493" quantity="2" label="2" />' + // DeepPink
      '<ColorMapEntry color="#C71585" quantity="2.5" label="2.5" />' + // MediumVioletRed
      '<ColorMapEntry color="#8B008B" quantity="6" label="6" />' + // DarkMagenta
    '</ColorMap>' +
  '</RasterSymbolizer>';

// Add MSI images to the map using the color classification
Map.addLayer(pre_fire_msi_2016.sldStyle(sld_intervals), {}, '2016 MSI classified');
Map.addLayer(post_fire_msi_2017.sldStyle(sld_intervals), {}, '2017 MSI classified');
Map.addLayer(post_fire_msi_2021.sldStyle(sld_intervals), {}, '2021 MSI classified');


// =======   ADD the MSI LEGEND  ======== //

// Create legend panel and set its position
var legend = ui.Panel({
  style: {
    position: 'bottom-left',
    padding: '8px 20px'
  }});
 
// Create legend title
var legendTitle = ui.Label({
  value: 'MSI Values',
  style: {fontWeight: 'bold',
    fontSize: '18px',
    margin: '0 0 6px 0',
    padding: '0'
    }});
 
// Add the title to the panel
legend.add(legendTitle);
 
// Creates and styles 1 row of the legend.
var makeRow = function(color, name) {
 
      // Create the label that is actually the colored box.
      var colorBox = ui.Label({
        style: {
          backgroundColor: '#' + color,
          // Use padding to give the box height and width.
          padding: '8px',
          margin: '0 0 4px 0'
        }});
 
      // Create the label filled with the description text.
      var description = ui.Label({
        value: name,
        style: {margin: '0 0 4px 6px'}
      });
 
      // Return the panel
      return ui.Panel({
        widgets: [colorBox, description],
        layout: ui.Panel.Layout.Flow('horizontal')
      })};
 
//  Palette with the colors
var palette =['006400', '228B22', '32CD32', '7FFFD4', 'ADFF2F', 'FFFFE0',
              'FFD700', 'FFC0CB', 'FF69B4', 'FF1493', 'C71585', '8B008B'];
 
// Name of the legend
var names = ['<0.2', '0.2 ÷ 0.4', '0.4 ÷ 0.6', '0.6 ÷ 0.8', '0.8 ÷ 1', '1 ÷ 1.2',
             '1.2 ÷ 1.4', '1.4 ÷ 1.6', '1.6 ÷ 1.8', '1.8 ÷ 2', '2 ÷ 2.5', '> 2.5'];
 
// Add color and and names to a legend panel
for (var i = 0; i < 12; i++) {
  legend.add(makeRow(palette[i], names[i]));
  }  
 
// Add legend to map (alternatively you can also print the legend to the console)
Map.add(legend);


// ======= Display Fire Boundary ======= //

// Display the fire boundary
Map.addLayer(fire_perimeter.style({width: 2,
                                  color: "red",
                                  fillColor: "#00000000"}),{},"Fire Boundary", 1);


// ======= Plot histogram charts ======= //

// Plot histogram charts for each MSI image in the console

plot_hist(pre_fire_msi_2016, studyArea, 50, 'MSI 2016');
plot_hist(post_fire_msi_2017, studyArea, 50, 'MSI 2017');
plot_hist(post_fire_msi_2021, studyArea, 50, 'MSI 2021');
```